# Fuzzy Logic Controller 

In [1]:
class Fuzzy:
    """
    Static class for defining Fuzzy Membership Functions.
    """
    @staticmethod
    def tri(x, a, b, c):
        """
        Triangular membership function.
        a: lower bound
        b: center/peak (membership value is 1)
        c: upper bound
        """
        # Outside the support range [a, c]
        if x <= a or x >= c:
            return 0
        
        # Left side of the triangle (rising slope)
        if x <= b:
            # Avoid division by zero if a == b, though typically b > a
            if (b - a) == 0:
                return 1.0
            return (x - a) / (b - a)
        
        # Right side of the triangle (falling slope)
        else:
            # Avoid division by zero if c == b
            if (c - b) == 0:
                return 1.0
            return (c - x) / (c - b)

class FuzzyController:
    """
    Implements the Fuzzy Logic Controller using Mamdani-style inference 
    and Centroid of Area (COA) for defuzzification (simplified to Weighted Average).
    """
    def __init__(self):
        # Center points for the output (defuzzification)
        self.speed_centers = {'off': 0, 'low': 30, 'medium': 60, 'high': 90}

    def fuzzify_temp(self, t):
        """Maps crisp temperature to fuzzy sets (cold, warm, hot)."""
        return {
            'cold': Fuzzy.tri(t, 0, 5, 15),
            'warm': Fuzzy.tri(t, 10, 17.5, 25),
            'hot':  Fuzzy.tri(t, 20, 27.5, 35)
        }

    def fuzzify_humidity(self, h):
        """Maps crisp humidity to fuzzy sets (low, medium, high)."""
        return {
            'low': Fuzzy.tri(h, 0, 20, 40),
            'medium': Fuzzy.tri(h, 30, 50, 70),
            'high': Fuzzy.tri(h, 60, 80, 100)
        }

    def rules(self, T, H):
        """
        Inference step: Apply rules using MIN operator (Mamdani inference).
        T and H are the fuzzy sets for Temp and Humidity.
        Returns a list of (output_level, strength) tuples.
        """
        return [
            # If Temp is Cold AND Humidity is Low, THEN Speed is OFF
            ('off',    min(T['cold'], H['low'])),
            
            # General comfort rules
            ('low',    min(T['warm'], H['low'])),
            ('medium', min(T['warm'], H['medium'])),
            
            # Higher heat rules (cooling needed)
            ('low',    min(T['hot'], H['low'])),
            ('medium', min(T['hot'], H['medium'])),
            
            # Highest heat and humidity (max cooling needed)
            ('high',   min(T['hot'], H['high']))
        ]

    def defuzzify(self, rules):
        """
        Defuzzification using the Weighted Average method (a common simplification of COA).
        """
        # Numerator: Sum of (Center of Output Level * Strength of Rule)
        num = sum(self.speed_centers[lvl] * val for lvl, val in rules)
        
        # Denominator: Sum of all rule strengths
        den = sum(val for _, val in rules)
        
        # Calculate Weighted Average (Fan Speed)
        return num / den if den else 0

    def control(self, temp, hum):
        """Main control flow: Fuzzify -> Rule Inference -> Defuzzify."""
        # 1. Fuzzification
        T, H = self.fuzzify_temp(temp), self.fuzzify_humidity(hum)
        
        # 2. Inference (Rule Firing)
        fired = self.rules(T, H)
        
        # 3. Defuzzification
        final_speed = self.defuzzify(fired)
        
        return final_speed, T, H, fired

# ---------------- OUTPUT / EXECUTION ----------------

print("\nFUZZY LOGIC TEMPERATURE AND FAN SPEED CONTROLLER\n")
controller = FuzzyController()

# Define test cases (Input Temperature, Input Humidity, Scenario Description)
tests = [
    (10, 30, "Cold room, low humidity"),
    (18, 45, "Warm room, medium humidity"),
    (28, 75, "Hot room, high humidity"),
    (5, 20, "Very cold, very dry"),
    (32, 90, "Very hot, very humid"),
]

# Print a summary table of results
print(f"{'Temp(°C)':<10} {'Humidity(%)':<12} {'Fan Speed(%)':<14} Scenario")
print("-" * 60)
for t, h, s in tests:
    sp, _, _, _ = controller.control(t, h)
    print(f"{t:<10} {h:<12} {sp:<14.2f} {s}")
print("-" * 60)

# Detailed Analysis Example
t, h = 28, 75
sp, T, H, fired = controller.control(t, h)

print(f"\nDetailed Analysis for {t}°C, {h}% humidity")
print("--- Fuzzification Results ---")
print(f"Temperature Fuzzy: {T}")
print(f"Humidity Fuzzy: {H}")

print("\n--- Rules Fired (non-zero) ---")
for lvl, val in fired:
    if val > 0:
        print(f"  {lvl.capitalize():<6} Rule Strength: {val:.4f} (Center: {controller.speed_centers[lvl]})")

print(f"\nFinal Calculated Fan Speed: {sp:.2f}%")
print("Final Status:",
    "OFF" if sp < 15 else "LOW" if sp < 45 else "MEDIUM" if sp < 75 else "HIGH")


FUZZY LOGIC TEMPERATURE AND FAN SPEED CONTROLLER

Temp(°C)   Humidity(%)  Fan Speed(%)   Scenario
------------------------------------------------------------
10         30           0.00           Cold room, low humidity
18         45           60.00          Warm room, medium humidity
28         75           90.00          Hot room, high humidity
5          20           0.00           Very cold, very dry
32         90           90.00          Very hot, very humid
------------------------------------------------------------

Detailed Analysis for 28°C, 75% humidity
--- Fuzzification Results ---
Temperature Fuzzy: {'cold': 0, 'warm': 0, 'hot': 0.9333333333333333}
Humidity Fuzzy: {'low': 0, 'medium': 0, 'high': 0.75}

--- Rules Fired (non-zero) ---
  High   Rule Strength: 0.7500 (Center: 90)

Final Calculated Fan Speed: 90.00%
Final Status: HIGH
